<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=195984006" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550.0,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650.0,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400.0,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100.0,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800.0,274.473999
...,...,...,...,...,...,...,...,...
2140843,2024-09-05 00:00:00,NEER-USD,0.034885,0.033663,0.038637,0.033297,15241.0,0.033663
2140844,2024-09-06 00:00:00,NEER-USD,0.033663,0.030551,0.033750,0.028334,80341.0,0.030551
2140845,2024-09-07 00:00:00,NEER-USD,0.030551,0.028451,0.034057,0.028451,109499.0,0.028451
2140846,2024-09-08 00:00:00,NEER-USD,0.028451,0.033962,0.037203,0.028184,110288.0,0.033962


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol yf-ravs5v'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:09<00:00,  2.08it/s]

2000


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1725919567


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2000/2000 [11:26<00:00,  2.91it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

1997

# Read a sample

In [12]:
jsonf = pd.read_json('/kaggle/working/crypto_data/BTC-USD.json')
jsondict = jsonf.to_dict()
timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
symbol = list(sym for s in range(len(timestamp)))
low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
dates = return_timestamp(timestamp)
tempdf = pd.DataFrame(
    {
    'dates':dates,
    'symbol':symbol,
    'open':open_,
    'close':close,
    'high':high,
    'low':low,
    'volume':volume,
    'adj_close':adj_close
    }
)
tempdf

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800,274.473999
...,...,...,...,...,...,...,...,...
3535,2024-09-05 00:00:00,BTC-USD,57971.703125,56160.488281,58300.582031,55712.453125,31030280656,56160.488281
3536,2024-09-06 00:00:00,BTC-USD,56160.191406,53948.753906,56976.109375,52598.699219,49361693566,53948.753906
3537,2024-09-07 00:00:00,BTC-USD,53949.085938,54139.687500,54838.144531,53740.070312,19061486526,54139.687500
3538,2024-09-08 00:00:00,BTC-USD,54147.933594,54841.566406,55300.859375,53653.757812,18268287531,54841.566406


# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}.json')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


100%|██████████| 2000/2000 [01:55<00:00, 17.32it/s]


In [15]:
megadf['symbol'].unique()

array(['BTC-USD', 'ETH-USD', 'USDT-USD', ..., 'XMON-USD', 'TROG-USD',
       'ETHO-USD'], dtype=object)

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2015-01-01 00:00:00', '2024-09-09 22:12:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume       float64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume              float64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550.0,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650.0,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400.0,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100.0,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800.0,274.473999
...,...,...,...,...,...,...,...,...
2138644,2024-09-05 00:00:00,ETHO-USD,0.007804,0.006347,0.008318,0.006042,70268.0,0.006347
2138645,2024-09-06 00:00:00,ETHO-USD,0.006347,0.006860,0.007803,0.006045,70166.0,0.006860
2138646,2024-09-07 00:00:00,ETHO-USD,0.006860,0.007045,0.007986,0.005761,76601.0,0.007045
2138647,2024-09-08 00:00:00,ETHO-USD,0.007045,0.006926,0.007992,0.005733,79452.0,0.006926


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550.0,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650.0,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400.0,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100.0,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800.0,274.473999
...,...,...,...,...,...,...,...,...
2138644,2024-09-05 00:00:00,ETHO-USD,0.007804,0.006347,0.008318,0.006042,70268.0,0.006347
2138645,2024-09-06 00:00:00,ETHO-USD,0.006347,0.006860,0.007803,0.006045,70166.0,0.006860
2138646,2024-09-07 00:00:00,ETHO-USD,0.006860,0.007045,0.007986,0.005761,76601.0,0.007045
2138647,2024-09-08 00:00:00,ETHO-USD,0.007045,0.006926,0.007992,0.005733,79452.0,0.006926


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 268M/268M [00:10<00:00, 26.7MB/s]


Upload successful: 1000_cryptos.csv (268MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)